In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
import datetime
import time
import math
import random
import warnings
import sweetviz as sv

# Functions

# Create function to create code_dict and extra_code_dict
def create_code_dict():
    """
    create_code_dict() reads two files "Code_to_text" and "Extra_codes" and creates two dictionaries: code_dict and extra_code_dict.
    The file "Code_to_text" contains codes and their corresponding text descriptions, which are used to populate the code_dict.
    The file "Extra_codes" contains additional codes and descriptions, which are used to populate the extra_code_dict.

    Returns:
    --------
    code_dict: A dictionary containing the codes and their corresponding text descriptions from "Code_to_text" file.
    extra_code_dict: A dictionary containing additional codes and their corresponding text descriptions from "Extra_codes" file.
    Note: The function adds an additional code '930C' with its corresponding text description 'Screaming child' to the code_dict.
    """

    f = open("Code_to_text", "r")
    code_dict = {}
    for x in f:
        temp = x[0 : -1].split(',')
        code_dict[temp[0]] = temp[1]
    code_dict['930C'] = 'Screaming child'
    f = open("Extra_codes", "r")
    extra_code_dict = {}
    for x in f:
        temp = x.replace('\t', '').replace('\n', '').split(',')
        extra_code_dict[temp[0]] = temp[1]
    return code_dict, extra_code_dict

# Create a function to merge all the csvs into one df and save
def merge_csvs(path_to_read, path_to_save):
    """
    Merge multiple CSV files containing LAPD calls for service data into a single CSV file.

    Parameters:
    path_to_read (str): The path to the directory containing the CSV files to merge.
    path_to_save (str): The path and filename of the merged CSV file to be saved.

    Returns:
    None

    The function searches for all CSV files in the specified directory with names starting with "LAPD_Calls_for_Service_",
    and reads all columns from each file into a list. It then converts the list of columns to a set to find the unique columns.
    Next, the function loads all files into a Pandas DataFrame and combines them.
    The function also renames some of the columns to ensure consistency across files.
    Finally, the function saves the combined DataFrame to a new CSV file specified by the path_to_save parameter.
    """
    # Find all the file names within LAPD Calls For Service
    # path_to_read = 'LAPD Calls For Service/'
    files = os.listdir(path_to_read)
    files = [file for file in files if file.endswith('.csv')]
    files = [file for file in files if file.startswith('LAPD_Calls_for_Service_')]
    files = sorted(files)


    # Read all files from path and print the columns for each file
    list_of_all_columns = []
    for file in files:
        df = pd.read_csv(path_to_read+file, low_memory=False)
        list_of_all_columns.append(list(df.columns))

    # Convert all columns to a set to find the unique columns
    set_of_all_columns = set()
    for column in list_of_all_columns:
        set_of_all_columns.update(column)


    # Load all files into df and combine
    df = pd.DataFrame()
    for file in files:
        temp_df = pd.read_csv(path_to_read + file)
        new_columns = []
        for column in temp_df.columns:
            if column == 'Area Occurred':
                new_columns.append('Area_Occ')
            elif column == 'Call Type Code':
                new_columns.append('Call_Type_Code')
            elif column == 'Call Type Description':
                new_columns.append('Call_Type_Text')
            elif column == 'Dispatch Date':
                new_columns.append('Dispatch_Date')
            elif column == 'Dispatch Time':
                new_columns.append('Dispatch_Time')
            elif column == 'Incident Number':
                new_columns.append('Incident_Number')
            elif column == 'Reporting District':
                new_columns.append('Rpt_Dist')
            else:
                new_columns.append(column)
        temp_df.columns = new_columns
        df = df.append(temp_df)
        df = df.reset_index(drop=True)

    df.to_csv(path_to_save)
    # df.to_csv('LAPD Calls For Service Combined.csv')
    return

# create a function to read the merged df
def read_and_clean(path_to_read):
    """
    Read and clean a combined CSV file containing LAPD calls for service data.

    Parameters:
    path_to_read : str
    The file path of the combined CSV file to read and clean.

    Returns:
    df : pandas DataFrame
    The cleaned DataFrame containing LAPD calls for service data. 

    Notes:
    The function drops the first column and 'Incident_Number' column, and converts the 'Dispatch_Date' column to a string
    of length 10. It replaces '^006' in the 'Call_Type_Code' column with '006'. It also converts all columns to string
    except 'Rpt_Dist', which is converted to float.
    """
    # df = pd.read_csv('LAPD Calls For Service Combined.csv', low_memory=False)
    df = pd.read_csv(path_to_read, low_memory=False)
    df = df.drop(df.columns[0], axis = 1)
    df.drop(['Incident_Number'], axis = 1, inplace = True)
    df['Dispatch_Date'] = df['Dispatch_Date'].apply(lambda x: x[0 : 10])
    # Replace ^006 in Call_Type_Code with '006'
    df['Call_Type_Code'] = df['Call_Type_Code'].apply(lambda x: x.replace('^006', '006'))

    # Convert all columns to string except Rpt_Dist
    for column in df.columns:
        if column == 'Rpt_Dist':
            df[column] = df[column].astype(float)
        else:
            df[column] = df[column].astype(str)
    
    return(df)

# Create a function for code mapping
def create_map(df, code_dict):
    """
    Creates a new DataFrame with two columns: Call_Type_Code and Call_Type_Text. 
    The Call_Type_Code is obtained from the given DataFrame, and the Call_Type_Text is determined 
    based on the longest word in the list of Call_Type_Text corresponding to the given Call_Type_Code.
    Then, Call_Type_Text is replaced by looking up the corresponding Call_Type_Code in code_dict.

    Parameters
    ----------
    df: pandas DataFrame
        The DataFrame to obtain Call_Type_Code and Call_Type_Text from.

    code_dict: dict
        The dictionary with Call_Type_Code as the key and the corresponding Call_Type_Text as the value.

    Returns
    -------
    pandas DataFrame
        A new DataFrame with two columns: Call_Type_Code and Call_Type_Text.
    """
    code_to_text_df = df[['Call_Type_Code', 'Call_Type_Text']].groupby(['Call_Type_Code'])['Call_Type_Text'].apply(list).reset_index(name='Call_Type_Text')
    # Replace Call_Type_Text with the longest word in the list
    code_to_text_df['Call_Type_Text'] = code_to_text_df['Call_Type_Text'].apply(lambda x: max(x, key=len))

    # Replace Call_Type_Text by looking up the corresponding Call_Type_Code in code_dict
    code_to_text_df['Call_Type_Text'] = code_to_text_df[['Call_Type_Code', 'Call_Type_Text']].apply(lambda x: code_dict.get(x[0], x[1]), axis = 1)
    return(code_to_text_df)

# create a function to get time related features
def create_time_features(df):
    """
    Extract various time features from the Dispatch_Date and Dispatch_Time columns of the input DataFrame.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame to process. It should contain the columns "Dispatch_Date" and "Dispatch_Time".

    Returns:
    --------
    pandas.DataFrame
        A new DataFrame with the following additional columns:
        - Dispatch_month_year: The month and year of the dispatch date, in the format "YYYY-MM".
        - Dispatch_month: The month of the dispatch date, in the format "MM".
        - Dispatch_year: The year of the dispatch date, in the format "YYYY".
        - Dispatch_Hour: The hour of the dispatch time, in the format "HH".
    """
# Finding time data from date
    df['Dispatch_month_year'] = pd.to_datetime(df['Dispatch_Date'].apply(lambda x: x[0 : 3] + x[-4 :]), format='%m/%Y').apply(lambda x: x.strftime('%Y-%m'))
    df['Dispatch_month'] = pd.to_datetime(df['Dispatch_Date'].apply(lambda x: x[0 : 2]), format='%m').apply(lambda x: x.strftime('%m'))
    df['Dispatch_year'] = pd.to_datetime(df['Dispatch_Date'].apply(lambda x: x[-4 :]), format='%Y').apply(lambda x: x.strftime('%Y'))
    df['Dispatch_Hour'] = pd.to_datetime(df['Dispatch_Time'], format='%H:%M:%S').apply(lambda x: x.strftime('%H'))
    return(df)

# Convert the codes to readable text
def convert_police_codes(df, code_to_text_df):
    """
    The convert_police_codes function takes in two parameters df and code_to_text_df. It converts the code_to_text_df DataFrame into a dictionary, removes non-digits from the Call_Type_Code column of df, and replaces them with the corresponding call type text based on the code_to_text_dict. The resulting DataFrame is returned.

    Parameters:
    df : Pandas DataFrame
    The DataFrame that contains the police code information.
    code_to_text_df : Pandas DataFrame
    The DataFrame that contains the mapping between police codes and their corresponding call type text.
    Returns:

    df : Pandas DataFrame
    The input DataFrame df with the Call_Type_Code column cleaned by replacing the codes with the corresponding call type text.
    """
    # convert code_to_text_df to dictionary 
    code_to_text_dict = code_to_text_df.set_index('Call_Type_Code')['Call_Type_Text'].to_dict()

    # Remove non digits from Call_Type_Code
    df['Call_Type_Code_Base'] = df['Call_Type_Code'].apply(lambda x: ''.join([i for i in x if i.isdigit()]))
    df['Call_Type_Code_Base_String'] = df['Call_Type_Code_Base'].apply(lambda x: code_to_text_dict.get(x, x))
    return(df)

Running Functions

In [2]:
code_dict, extra_code_dict = create_code_dict()

In [3]:
merge_csvs('LAPD Calls For Service/', 'temp.csv')

In [4]:
df = read_and_clean('temp.csv')
df

,Rpt_Dist,Area_Occ,Dispatch_Date,Dispatch_Time,Call_Type_Code,Call_Type_Text
0,1184.0,Northeast,12/31/2010,23:59:41,507P,PARTY
1,1367.0,Newton,12/31/2010,23:59:39,9212N,TRESPASS SUSP NOW
2,147.0,Central,12/31/2010,23:59:08,930W,WMN
3,441.0,Hollenbeck,12/31/2010,23:58:57,246H,HEARD ONLY
4,514.0,Harbor,12/31/2010,23:58:51,242DS,DOM VIOL SUSP
...,...,...,...,...,...,...
15473575,1983.0,Mission,01/01/2023,00:03:47,242S,242 SUSP
15473576,701.0,Wilshire,01/01/2023,00:02:14,9065,906 VISUAL (C30 VICT
15473577,1781.0,Devonshire,01/01/2023,00:02:08,006,CODE 6
15473578,NaN,Outside,01/01/2023,00:02:07,006,CODE 6


In [5]:
code_to_text_df = create_map(df, code_dict)
code_to_text_df

,Call_Type_Code,Call_Type_Text
0,006,CODE 6
1,110,110 NARCOTICS
2,1101,Narcotic activity
3,1101Q,NARCOTIC ACT BUS
4,1101U,NARCOTIC ACT U/R
...,...,...
1037,997,997 SUSPICIOUS ACTIV
1038,999,999 HELP
1039,9991,999 OFCR NEEDS HELP
1040,999F,999 FD HELP


In [6]:
df = create_time_features(df)
df

,Rpt_Dist,Area_Occ,Dispatch_Date,Dispatch_Time,Call_Type_Code,Call_Type_Text,Dispatch_month_year,Dispatch_month,Dispatch_year,Dispatch_Hour
0,1184.0,Northeast,12/31/2010,23:59:41,507P,PARTY,2010-12,12,2010,23
1,1367.0,Newton,12/31/2010,23:59:39,9212N,TRESPASS SUSP NOW,2010-12,12,2010,23
2,147.0,Central,12/31/2010,23:59:08,930W,WMN,2010-12,12,2010,23
3,441.0,Hollenbeck,12/31/2010,23:58:57,246H,HEARD ONLY,2010-12,12,2010,23
4,514.0,Harbor,12/31/2010,23:58:51,242DS,DOM VIOL SUSP,2010-12,12,2010,23
...,...,...,...,...,...,...,...,...,...,...
15473575,1983.0,Mission,01/01/2023,00:03:47,242S,242 SUSP,2023-01,01,2023,00
15473576,701.0,Wilshire,01/01/2023,00:02:14,9065,906 VISUAL (C30 VICT,2023-01,01,2023,00
15473577,1781.0,Devonshire,01/01/2023,00:02:08,006,CODE 6,2023-01,01,2023,00
15473578,NaN,Outside,01/01/2023,00:02:07,006,CODE 6,2023-01,01,2023,00


In [7]:
df = convert_police_codes(df, code_to_text_df)
df

,Rpt_Dist,Area_Occ,Dispatch_Date,Dispatch_Time,Call_Type_Code,Call_Type_Text,Dispatch_month_year,Dispatch_month,Dispatch_year,Dispatch_Hour,Call_Type_Code_Base,Call_Type_Code_Base_String
0,1184.0,Northeast,12/31/2010,23:59:41,507P,PARTY,2010-12,12,2010,23,507,507 MINOR DISTURBANC
1,1367.0,Newton,12/31/2010,23:59:39,9212N,TRESPASS SUSP NOW,2010-12,12,2010,23,9212,Trespass suspect
2,147.0,Central,12/31/2010,23:59:08,930W,WMN,2010-12,12,2010,23,930,930 SCREAMING
3,441.0,Hollenbeck,12/31/2010,23:58:57,246H,HEARD ONLY,2010-12,12,2010,23,246,246 SHOTS FIRED
4,514.0,Harbor,12/31/2010,23:58:51,242DS,DOM VIOL SUSP,2010-12,12,2010,23,242,242 BATTERY
...,...,...,...,...,...,...,...,...,...,...,...,...
15473575,1983.0,Mission,01/01/2023,00:03:47,242S,242 SUSP,2023-01,01,2023,00,242,242 BATTERY
15473576,701.0,Wilshire,01/01/2023,00:02:14,9065,906 VISUAL (C30 VICT,2023-01,01,2023,00,9065,906 VISUAL (C30 VICT
15473577,1781.0,Devonshire,01/01/2023,00:02:08,006,CODE 6,2023-01,01,2023,00,006,CODE 6
15473578,NaN,Outside,01/01/2023,00:02:07,006,CODE 6,2023-01,01,2023,00,006,CODE 6


In [8]:
df.to_csv('Cleaned Data Services.csv')